In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import torch.autograd as autograd
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from logReg import LogitReg
from torch.nn import Parameter
from influence import *
from util import *
import hessian as hess
from pathlib import Path
import time
from experiments import *

In [4]:
'''training_data = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True),shuffle=True)
test_data =  torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True),shuffle=True)



X_train = training_data.dataset.train_data.data.reshape(-1,784).type(torch.FloatTensor)/255
y_train = training_data.dataset.train_labels.data
X_test = test_data.dataset.test_data.data.reshape(-1,784).type(torch.FloatTensor)/255
y_test = test_data.dataset.test_labels


np.savez("../data/training_data.npz", X=X_train, Y= y_train)
np.savez("../data/test_data.npz", X=X_test, Y= y_test)'''

X_train = torch.from_numpy(np.load("../data/training_data.npz")["X"])
y_train = torch.from_numpy(np.load("../data/training_data.npz")["Y"])
X_test = torch.from_numpy(np.load("../data/test_data.npz")["X"])
y_test = torch.from_numpy(np.load("../data/test_data.npz")["Y"])



max_iter = 100
D_in = X_train.shape[1]
D_out = 10
N = X_train.shape[0]
weight_decay = 0.01

cls = LogitReg(max_iter, D_in, D_out, N, weight_decay)
cls.fit(X_train,y_train)
print("Training loss {}".format(cls.loss_fn(X_train, y_train)))
print( "Training accuracy {}". format( cls.score(X_train,y_train)))
print( "Test loss {}". format( cls.loss_fn(X_test,y_test)))
print( "Test accuracy {}". format( cls.score(X_test,y_test)))


LBFGS training took [67] iter.
After training with LBFGS: 
Training loss 0.41001251339912415
a 53993.0 b 60000
Training accuracy 0.8998833298683167
Test loss 0.3923059105873108
a 9056.0 b 10000
Test accuracy 0.9056000113487244


In [9]:
def hessian_k_leave_out( model, X_train, y_train, X_test, y_test, n_trials=1, num_to_remove=5, verify_influence = False, 
                         scale=25, depth=5000, batch_size = 50, num_samples=10):
       
        N = model.num_train_examples
        
        predicted = model.predict(X_test)
        test_indices = torch.nonzero(predicted != y_test)[0:n_trials]
        params_diff = np.zeros((n_trials, 3))
        loss_diff = np.zeros((n_trials, 3))
        time_diff = np.zeros((n_trials,2))
        train_indices = np.zeros((n_trials,num_to_remove))
        
        h_ncg = hess.Hessian( model, X_train, y_train )
        h_ncg.initialize(X_train, y_train)
        
        h_lissa = hess.Hessian( model, X_train, y_train )
        
        
        for trial in range(n_trials):
            np.random.seed()
            indices_to_remove = np.random.choice(N, num_to_remove)
            
            train_indices[trial,:] = indices_to_remove
            test_idx = test_indices[trial]
           
            X_tr, y_tr, X_te, y_te = split_data_k( indices_to_remove, test_idx, X_train, y_train, X_test, y_test)

            tr_grad, train_loss = h_ncg.get_loss_gradient(X_tr, y_tr)
            test_grad, test_loss = h_ncg.get_loss_gradient(X_te, y_te)
            
            v = test_grad.detach().numpy()
            
            print("Calculating HVP inverse using ncg")
            t0 = time.time()
            hvp_ncg = h_ncg.get_inverse_hvp_cg(v, max_iterations = 100)
            t1 = time.time()
            
            time_diff[trial,0] = t1-t0
            
            
            params_diff[trial, 0] = np.linalg.norm(np.array(hvp_ncg) * (num_to_remove/N))
            loss_diff[trial, 0] = np.sum(hvp_ncg * tr_grad.detach().numpy())*(num_to_remove/N)
            
            print("Calculating HVP inverse using lissa")
            
            t2 = time.time()
            hvp_lissa = stochastic_hessian_Lissa(h_lissa, X_train, y_train, v, scale=scale, 
                                                 depth=depth, batch_size = batch_size, num_samples= num_samples)
            
            t3 = time.time() 
            
            time_diff[trial,1] = t3-t2
            
  
            params_diff[trial, 1] = np.linalg.norm(np.array(hvp_lissa) * (num_to_remove/N))
            loss_diff[trial, 1] = np.sum(hvp_lissa * tr_grad.detach().numpy())*(num_to_remove/N)

            if( verify_influence):
                cls_leave = LogitReg(model.iterations, model.D_in, model.D_out, N, model.weight_decay)
                number = np.array(range(N))
                number = np.delete(number, indices_to_remove)
                
                cls_leave.fit( X_train[number,:], y_train[number])
                actual_difference = cls_leave.fc1.weight.view(-1) - model.fc1.weight.view(-1)
                 
                params_diff[trial,2] = torch.norm(actual_difference).detach().numpy()
                loss_diff[trial,2] =  cls_leave.loss_fn(X_te, y_te) - model.loss_fn(X_te, y_te)
                print( "Predicted loss  ncg{} liss {} actual loss {}".format(loss_diff[trial,0]
                                                                            ,loss_diff[trial,1],
                                                                             loss_diff[trial,2]))

        np.savez("../data/computation/hess_diffs_" + str(num_to_remove) + ".npz",r = {"params":params_diff, 
                                                                                      "loss":loss_diff, 
                                                                                      "test_indices": test_indices,
                                                                                      "train_indices": train_indices, 
                                                                                      "K":num_to_remove,
                                                                                      "time_diff":time_diff,
                                                                    })


In [ ]:
count = 1000
while( count >= 10 ):
    print("Leave-out-k-{}".format(count))
    hessian_k_leave_out( cls, X_train, y_train, X_test, y_test, n_trials = 100, num_to_remove=count, verify_influence = True, 
                     scale=25, depth=5000, batch_size = 1, num_samples=10)
    count = count - 100

Leave-out-k-1000
Calculating HVP inverse using ncg
         Current function value: -1712.486084
         Iterations: 9
         Function evaluations: 75
         Gradient evaluations: 72
         Hessian evaluations: 75
Calculating HVP inverse using lissa
(7840,)
Recursion at depth 0: norm is 24.07050705
Recursion at depth 1000: norm is 5647.08789062
Recursion at depth 2000: norm is 7945.43750000
Recursion at depth 3000: norm is 9170.41503906
Recursion at depth 4000: norm is 10208.51953125
Recursion at depth 4999: norm is 11051.63183594
(7840,)
Recursion at depth 0: norm is 24.12347221
Recursion at depth 1000: norm is 5608.37255859
Recursion at depth 2000: norm is 8251.64843750
Recursion at depth 3000: norm is 9775.54589844
Recursion at depth 4000: norm is 9749.49316406
Recursion at depth 4999: norm is 10309.54687500
(7840,)
Recursion at depth 0: norm is 24.14447212
Recursion at depth 1000: norm is 5297.74462891
Recursion at depth 2000: norm is 7945.34326172
Recursion at depth 3000: n

Recursion at depth 4999: norm is 4935.53857422
(7840,)
Recursion at depth 0: norm is 13.08242607
Recursion at depth 1000: norm is 2561.29736328
Recursion at depth 2000: norm is 4383.61376953
Recursion at depth 3000: norm is 4947.01562500
Recursion at depth 4000: norm is 4697.91845703
Recursion at depth 4999: norm is 5018.78906250
(7840,)
Recursion at depth 0: norm is 13.08250523
Recursion at depth 1000: norm is 2429.49169922
Recursion at depth 2000: norm is 4438.42675781
Recursion at depth 3000: norm is 4274.15771484
Recursion at depth 4000: norm is 5141.54882812
Recursion at depth 4999: norm is 5647.19824219
(7840,)
Recursion at depth 0: norm is 13.08224678
Recursion at depth 1000: norm is 2739.11816406
Recursion at depth 2000: norm is 3478.84155273
Recursion at depth 3000: norm is 4205.40185547
Recursion at depth 4000: norm is 5320.81005859
Recursion at depth 4999: norm is 4534.65576172
(7840,)
Recursion at depth 0: norm is 13.00465679
Recursion at depth 1000: norm is 2797.98364258
R

Recursion at depth 1000: norm is 4278.78320312
Recursion at depth 2000: norm is 6826.92089844
Recursion at depth 3000: norm is 8201.03222656
Recursion at depth 4000: norm is 8887.41796875
Recursion at depth 4999: norm is 9295.73535156
Function value at iter 0 and estimate -1463.8575439453125
LBFGS training took [68] iter.
After training with LBFGS: 
Predicted loss  ncg0.008331112066904704 liss 0.007994356751441955 actual loss 0.005391836166381836
Calculating HVP inverse using ncg
         Current function value: -311.131165
         Iterations: 9
         Function evaluations: 120
         Gradient evaluations: 115
         Hessian evaluations: 74
Calculating HVP inverse using lissa
(7840,)
Recursion at depth 0: norm is 12.35270596
Recursion at depth 1000: norm is 2552.16552734
Recursion at depth 2000: norm is 3574.88818359
Recursion at depth 3000: norm is 3696.53735352
Recursion at depth 4000: norm is 4105.80468750
Recursion at depth 4999: norm is 4310.03222656
(7840,)
Recursion at de

Recursion at depth 3000: norm is 4112.68066406
Recursion at depth 4000: norm is 4458.47265625
Recursion at depth 4999: norm is 4767.79296875
(7840,)
Recursion at depth 0: norm is 14.46019459
Recursion at depth 1000: norm is 2502.04248047
Recursion at depth 2000: norm is 3829.25585938
Recursion at depth 3000: norm is 4243.73242188
Recursion at depth 4000: norm is 4669.22070312
Recursion at depth 4999: norm is 4859.57373047
(7840,)
Recursion at depth 0: norm is 14.56340694
Recursion at depth 1000: norm is 2481.37988281
Recursion at depth 2000: norm is 3549.47070312
Recursion at depth 3000: norm is 4220.54492188
Recursion at depth 4000: norm is 4404.40820312
Recursion at depth 4999: norm is 4336.58496094
(7840,)
Recursion at depth 0: norm is 14.21666431
Recursion at depth 1000: norm is 2246.80297852
Recursion at depth 2000: norm is 3767.31982422
Recursion at depth 3000: norm is 4189.78369141
Recursion at depth 4000: norm is 4485.08007812
Recursion at depth 4999: norm is 4795.57275391
(784

Recursion at depth 4000: norm is 8907.69042969
Recursion at depth 4999: norm is 9493.67285156
(7840,)
Recursion at depth 0: norm is 14.79082870
Recursion at depth 1000: norm is 4503.79248047
Recursion at depth 2000: norm is 6919.79589844
Recursion at depth 3000: norm is 8115.98193359
Recursion at depth 4000: norm is 9062.10253906
Recursion at depth 4999: norm is 9722.16699219
(7840,)
Recursion at depth 0: norm is 14.97955894
Recursion at depth 1000: norm is 4518.75830078
Recursion at depth 2000: norm is 6939.97167969
Recursion at depth 3000: norm is 8272.17285156
Recursion at depth 4000: norm is 8943.56933594
Recursion at depth 4999: norm is 9575.90429688
Function value at iter 0 and estimate -1476.795166015625
LBFGS training took [66] iter.
After training with LBFGS: 
Predicted loss  ncg-0.0077719549338022866 liss -0.012381172180175782 actual loss 0.0008168220520019531
Calculating HVP inverse using ncg
         Current function value: -601.473511
         Iterations: 9
         Functi

...

Recursion at depth 2000: norm is 6030.79345703
Recursion at depth 3000: norm is 7517.87939453
Recursion at depth 4000: norm is 8676.53027344
Recursion at depth 4999: norm is 9076.17871094
Function value at iter 0 and estimate -810.496337890625
LBFGS training took [67] iter.
After training with LBFGS: 
Predicted loss  ncg0.0022147521376609802 liss 0.004714974761009216 actual loss 0.0069206953048706055
Calculating HVP inverse using ncg
         Current function value: -638.808472
         Iterations: 9
         Function evaluations: 86
         Gradient evaluations: 83
         Hessian evaluations: 81
Calculating HVP inverse using lissa
(7840,)
Recursion at depth 0: norm is 13.17734432
Recursion at depth 1000: norm is 3634.62255859
Recursion at depth 2000: norm is 5019.33447266
Recursion at depth 3000: norm is 5794.99072266
Recursion at depth 4000: norm is 6223.56738281
Recursion at depth 4999: norm is 6616.47314453
(7840,)
Recursion at depth 0: norm is 13.26459026
Recursion at depth 100

Recursion at depth 4000: norm is 8218.43847656
Recursion at depth 4999: norm is 8672.25976562
(7840,)
Recursion at depth 0: norm is 22.98111343
Recursion at depth 1000: norm is 4584.09912109
Recursion at depth 2000: norm is 6305.60937500
Recursion at depth 3000: norm is 7768.72558594
Recursion at depth 4000: norm is 8524.82421875
Recursion at depth 4999: norm is 8667.10449219
(7840,)
Recursion at depth 0: norm is 22.96998215
Recursion at depth 1000: norm is 4713.41455078
Recursion at depth 2000: norm is 6501.39062500
Recursion at depth 3000: norm is 7625.57373047
Recursion at depth 4000: norm is 8160.15527344
Recursion at depth 4999: norm is 8505.85546875
(7840,)
Recursion at depth 0: norm is 23.17340851
Recursion at depth 1000: norm is 4625.82373047
Recursion at depth 2000: norm is 6208.59130859
Recursion at depth 3000: norm is 7235.00000000
Recursion at depth 4000: norm is 8042.82812500
Recursion at depth 4999: norm is 8355.24707031
(7840,)
Recursion at depth 0: norm is 23.22473335
R

Recursion at depth 4999: norm is 9930.89355469
(7840,)
Recursion at depth 0: norm is 21.92612267
Recursion at depth 1000: norm is 5592.57031250
Recursion at depth 2000: norm is 8404.71875000
Recursion at depth 3000: norm is 9232.65625000
Recursion at depth 4000: norm is 9623.59765625
Recursion at depth 4999: norm is 10698.58398438
(7840,)
Recursion at depth 0: norm is 21.85085678
Recursion at depth 1000: norm is 5307.38183594
Recursion at depth 2000: norm is 8156.90625000
Recursion at depth 3000: norm is 9502.58789062
Recursion at depth 4000: norm is 9979.22265625
Recursion at depth 4999: norm is 10501.29101562
Function value at iter 0 and estimate -2104.4593505859375
LBFGS training took [66] iter.
After training with LBFGS: 
Predicted loss  ncg0.002247822284698486 liss 0.001108461618423462 actual loss 0.007909536361694336
Calculating HVP inverse using ncg
         Current function value: -293.118134
         Iterations: 9
         Function evaluations: 85
         Gradient evaluations

Recursion at depth 1000: norm is 3373.35083008
Recursion at depth 2000: norm is 5111.38769531
Recursion at depth 3000: norm is 5530.33398438
Recursion at depth 4000: norm is 6489.27441406
Recursion at depth 4999: norm is 6600.04248047
(7840,)
Recursion at depth 0: norm is 21.95458984
Recursion at depth 1000: norm is 3239.49414062
Recursion at depth 2000: norm is 4854.27490234
Recursion at depth 3000: norm is 5848.74121094
Recursion at depth 4000: norm is 6113.51220703
Recursion at depth 4999: norm is 6256.60498047
(7840,)
Recursion at depth 0: norm is 21.83825111
Recursion at depth 1000: norm is 3520.28759766
Recursion at depth 2000: norm is 4853.05761719
Recursion at depth 3000: norm is 5818.89013672
Recursion at depth 4000: norm is 6128.31103516
Recursion at depth 4999: norm is 6581.40966797
(7840,)
Recursion at depth 0: norm is 21.88606071
Recursion at depth 1000: norm is 3527.07128906
Recursion at depth 2000: norm is 4824.53613281
Recursion at depth 3000: norm is 5354.21191406
Recu

Recursion at depth 1000: norm is 5309.36914062
Recursion at depth 2000: norm is 7336.78808594
Recursion at depth 3000: norm is 8660.31640625
Recursion at depth 4000: norm is 9522.89453125
Recursion at depth 4999: norm is 9766.74316406
(7840,)
Recursion at depth 0: norm is 22.52882767
Recursion at depth 1000: norm is 5505.66894531
Recursion at depth 2000: norm is 7797.90917969
Recursion at depth 3000: norm is 8714.74707031
Recursion at depth 4000: norm is 9497.13671875
Recursion at depth 4999: norm is 10391.59960938
(7840,)
Recursion at depth 0: norm is 21.82168007
Recursion at depth 1000: norm is 5311.51220703
Recursion at depth 2000: norm is 7331.52148438
Recursion at depth 3000: norm is 8771.88085938
Recursion at depth 4000: norm is 9756.17382812
Recursion at depth 4999: norm is 10338.26074219
(7840,)
Recursion at depth 0: norm is 22.59743881
Recursion at depth 1000: norm is 5491.87207031
Recursion at depth 2000: norm is 7708.95947266
Recursion at depth 3000: norm is 8464.58886719
Re